In [256]:
import streamlit as st
import pandas as pd
import numpy as np
import re
import ast
import glob
from datetime import datetime
from fuzzywuzzy import fuzz

In [257]:
data = pd.read_csv('data/structured_reports/Sample_annotated_report_database.csv', encoding="iso-8859-1")

In [154]:
text_test = data['Patient_report'].iloc[8]

In [258]:
text_test = data['Patient_report'].iloc[31]

In [259]:
text_test

'CLINICAL HISTORY:  38 year old right handed female with seizures since she was 3 years of age, now with increased seizure frequency, here for presurgical evaluation.\n\nMEDICATIONS:  Lorazepam, Keppra, Oxcarbazepine, Lyrica, Zonisamide.\n\nREASON FOR STUDY:  Seizures\n\nINTRODUCTION: Digital video EEG was performed in lab using standard 10-20 electrode placement system with additional anterior temporal and single-lead EKG electrodes.  The patient was recorded in wakefulness and sleep.  Activating procedures were performed including hyperventilation and photic stimulation.\n\nTECHNICAL DIFFICULTIES:  None.\n\nDESCRIPTION OF THE RECORD:  The record opens to a well-defined posterior dominant rhythm of 9 to 10 Hz, which is seen on both sides and is reactive to eye opening.  There is a normal amount of frontocentral beta.  The patient is recorded in wakefulness and stage I and stage II sleep.  Activating procedure produce no abnormal discharges.\n\nABNORMAL DISCHARGES:\n1. Focal sharp wave

In [260]:
tags_targets

array(['BFNE', 'EME', 'Ohtahara', 'Myoclonic encephalopathy', 'infantile',
       'migrating focal', 'MEI', 'Dravet', 'West', 'myoclonic atonic',
       'Gastaut type', 'CAE', 'myoclonic absences', 'Landau-Kleffner',
       'Lennox-Gastaut', 'Panayiotopoulos', 'tonic-clonic', 'jAE',
       'ADEAF', 'JME', 'Gelastic', 'Reflex', 'Rasmussen', 'central',
       'supplementary motor area', 'Temporoparietal junction', 'frontal',
       'fontral temporal', 'insular', 'multifocal', 'occipital',
       'parietal', 'temporal lobe', 'external temporal', 'MTLE with HS',
       'temporal occipital', 'temporal'], dtype=object)

In [261]:
def load_tags_list():
    data = pd.read_csv('data/parameters/tags_list.csv', encoding="iso-8859-1")
    tags_list = data['tags'].tolist()
    return tags_list

In [262]:
tags_list = load_tags_list()

In [263]:
tags_list

['BFNE',
 'EME',
 'Ohtahara',
 'Myoclonic encephalopathy',
 'infantile',
 'migrating focal',
 'MEI',
 'Dravet',
 'West',
 'myoclonic atonic',
 'Gastaut type',
 'CAE',
 'myoclonic absences',
 'Landau-Kleffner',
 'Lennox-Gastaut',
 'Panayiotopoulos',
 'tonic-clonic',
 'jAE',
 'ADEAF',
 'JME',
 'Gelastic',
 'Reflex',
 'Rasmussen',
 'central',
 'supplementary motor area',
 'Temporoparietal junction',
 'frontal',
 'fontral temporal',
 'insular',
 'multifocal',
 'occipital',
 'parietal',
 'temporal lobe',
 'external temporal',
 'MTLE with HS',
 'temporal occipital',
 'temporal']

tags_list = pd.read_csv('data/parameters/tags_list.csv', encoding="iso-8859-1")
tags_list = tags_list['tags'].values
tags_targets = tags_list
print(tags_list)

In [264]:
def surline_keyword_info_extractor(text, tags_targets, threshold):
    keyword_list = []
    targets_list = []
    position_list = []
    for tag_target in tags_targets:
        for i in text.split('.'):
                if fuzz.partial_ratio(i.lower(), tag_target.lower()) >= threshold:
                    for j in re.split(';|,|:| |:|-',i):
                        # CONFIRM PARTIAL RATIO
                        if fuzz.partial_ratio(j.lower(), tag_target.lower()) >= threshold and len(j)>4:
                            keyword_list.append(tag_target)
                            targets_list.append(j)
    
    return list(set(keyword_list)), list(set(targets_list))

In [265]:

 keyword_list, targets_list= surline_keyword_info_extractor(text_test, tags_targets, 90)
 print(keyword_list)
 print(targets_list)

['central', 'temporal lobe', 'EME', 'temporal']
['frontocentral', 'temporal', 'placement']


In [266]:
def levenshtein_extraction(report, tags_targets, threshold):
    keyword_list = []
    targets_list = []
    position_list = []
    for tag_target in tags_targets:
        for i in report.split('.'):
                if fuzz.partial_ratio(i.lower(), tag_target.lower()) >= threshold:
                    for j in re.split(';|,|:| |:|-',i):
                        # CONFIRM PARTIAL RATIO
                        if fuzz.partial_ratio(j.lower(), tag_target.lower()) >= threshold and len(j)>4:
                            keyword_list.append(tag_target)
                            targets_list.append(j)
    keyword_list = list(set(keyword_list))
    targets_list = list(set(targets_list))
    
    return keyword_list, targets_list

In [267]:
keyword_list, targets_list = levenshtein_extraction(text_test, tags_list, 90)
print(keyword_list)
print(targets_list)

['central', 'temporal lobe', 'EME', 'temporal']
['frontocentral', 'temporal', 'placement']


In [268]:
def create_highlighted_markdown_text(report, tags_list):
    #try :
    # Initialization and parameters

    # TO DELETE

   # if pd.isnull(highlighted_information):
   #     report = re.sub("\n", "<br>", report)
   #     return report

    # Creating a dataset of the elements to highlight
    #df_highlighted_batch = pd.DataFrame(columns=['word','coordinates'])

   # for highlighted_info in ast.literal_eval(highlighted_information):
   #     words = highlighted_info[0]
   #     index = highlighted_info[1]
   #     for i in range(len(words)):
  #          df_highlighted_batch = df_highlighted_batch.append({'word':words[i], 'coordinates':index[i]}, ignore_index=True)
   # df_highlighted_batch.sort_values(by="coordinates", inplace=True) # for highlighting in order of position

    # Updating the report
    #for i in range(df_highlighted_batch.shape[0]):
     #   word = df_highlighted_batch['word'].iloc[i]
      #  index = df_highlighted_batch['coordinates'].iloc[i]

     #   decorate_word = html_decorate_text(word, background_color="#FFFF00")
     #   report = report[:index + shift] + decorate_word + report[len(report[:index + shift] + word):]
     #   shift += len(decorate_word) - len(word)
    #n = df_highlighted_batch.shape[0]
    #for i in range(n):
    #    word = df_highlighted_batch['word'].iloc[n-i-1]
    #    index = df_highlighted_batch['coordinates'].iloc[n-i-1]
    #    decorate_word = html_decorate_text(word, background_color="#FFFF00")
    #    report = report[:index] + decorate_word + report[len(report[:index] + word):]

    # changing for regex
     #word_list =  df_highlighted_batch['word'].unique()
     #word_list.sort()    
    # To prevent double <>
     #for word in word_list:
       #   decorate_word = html_decorate_text(word, background_color="#FFFF00")
        # report = re.sub(' '+word+' ', ' '+decorate_word+' ', report) 


    # Keep newline in the markdown report

    keyword_list, targets_list = levenshtein_extraction(report, tags_list, 90)
    #testing
    report = neutral_tags_addition(report, targets_list)

    for word in targets_list:
        decorate_word = html_decorate_text(word, background_color="#FFFF00")
        report = re.sub(' '+word+' ', ' '+decorate_word+' ', report) 

    report = re.sub("\n", "<br>", report)
    report = bolded_tagged_sentenced(report)
    
    return report

In [269]:
tags_list

['BFNE',
 'EME',
 'Ohtahara',
 'Myoclonic encephalopathy',
 'infantile',
 'migrating focal',
 'MEI',
 'Dravet',
 'West',
 'myoclonic atonic',
 'Gastaut type',
 'CAE',
 'myoclonic absences',
 'Landau-Kleffner',
 'Lennox-Gastaut',
 'Panayiotopoulos',
 'tonic-clonic',
 'jAE',
 'ADEAF',
 'JME',
 'Gelastic',
 'Reflex',
 'Rasmussen',
 'central',
 'supplementary motor area',
 'Temporoparietal junction',
 'frontal',
 'fontral temporal',
 'insular',
 'multifocal',
 'occipital',
 'parietal',
 'temporal lobe',
 'external temporal',
 'MTLE with HS',
 'temporal occipital',
 'temporal']

In [186]:
def neutral_tags_addition(report, neutral_tags_list):
    updated_report = report
    for neutral_tags in neutral_tags_list:
        search = neutral_tags
        updated_report = re.sub(search, html_decorate_text(neutral_tags, background_color='#00ecff'), updated_report)
     
        #search = '(\w*)?({})\w*'.format(neutral_tags)
        #updated_report = re.sub(neutral_tags, html_decorate_text(neutral_tags, background_color='#00ecff'), updated_report)

      # with capital letter
        #maj_neutral_tags = str(neutral_tags[:1].upper()+neutral_tags[1:])
        #search = maj_neutral_tags
        #search = '\w*({})\w*'.format(maj_neutral_tags)
        #updated_report = re.sub(search, html_decorate_text(neutral_tags, background_color='#00ecff'), updated_report)
    return updated_report

In [270]:
data

,Exam_name,Patient_name,Patient_infos_filepath,Quality_score,Exam_duration,Nb_Seizures,Recording_name,Patient_report,Tags,Seizure_type,Highlighted_data
0,00000006_s004,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,0,00000006_s004_t000,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
1,00000006_s004,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,0,00000006_s004_t001,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
2,00000006_s004,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,0,00000006_s004_t002,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
3,00000006_s005,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,1,00000006_s005_t000,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
4,00000006_s006,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,0,00000006_s006_t000,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
5,00000006_s006,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,0,00000006_s006_t001,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
6,00000006_s006,6,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.000000,0,0,00000006_s006_t002,"Long-Term Monitoring, February 2, 2007 to Febr...",NaN,NaN,NaN
7,00000258_s001,258,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.999340,1750,6,00000258_s001_t000,CLINICAL HISTORY: This is a 41-year-old male w...,NaN,NaN,NaN
8,00000258_s002,258,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.852459,20,0,00000258_s002_t000,CLINICAL HISTORY: This is a 41-year-old male w...,"tonic-clonic, temporal","Generalized, focal","[(['tonic-clonic'], [2061]), (['temporal'], [2..."
9,00000258_s002,258,./../TU_EEG_Seizures_processed/v0_4/info-patie...,0.374269,272,0,00000258_s002_t002,CLINICAL HISTORY: This is a 41-year-old male w...,"tonic-clonic, temporal","Generalized, focal","[(['tonic-clonic'], [2061]), (['temporal'], [2..."


In [254]:
text_test = data['Patient_report'].iloc[3]
create_highlighted_markdown_text(text_test, tags_list)

KeyError: 'Patient_report'

In [271]:
def html_decorate_text(text, background_color = "#DDDDDD", font_weight = "500"):
    return '<span style="background-color: '+ background_color +'; font-weight: '+ font_weight +';">'+ text +'</span>'

def bolded_tagged_sentenced(report):
    bolded_report = ''
    for sentence in str(report).split('.'):
        if re.search('<span style=', sentence):
            sentence = str('**') + sentence + str('**.') 
        else:
            sentence = sentence + '.'
        bolded_report += sentence
    return bolded_report

In [272]:
def html_decorate_tag_list(tag_list):
    if len(tag_list) == 0:
        return tag_list
    else:
        tag_list_content = [html_decorate_text(content) for content in tag_list]
        tag_list_content = ", ".join(tag_list_content)
        return tag_list_content

In [273]:
html_decorate_tag_list(targets_list)

'<span style="background-color: #DDDDDD; font-weight: 500;">frontocentral</span>, <span style="background-color: #DDDDDD; font-weight: 500;">temporal</span>, <span style="background-color: #DDDDDD; font-weight: 500;">placement</span>'

In [274]:
data_neutral = pd.read_csv('data/parameters/simplified_key_words.csv', encoding="utf-8")

In [275]:
data_neutral.head()

,symptome-en-simple,type of crisis
0,absences,NaN
1,ADEAF,NaN
2,atonic,Generalized
3,BFNE,NaN
4,CAE,Generalized


In [278]:
targets_list

['frontocentral', 'temporal', 'placement']

In [287]:

def crisis_type_correspondance(target_tags_list, correspondance_dataset):
    crisis_type_list = []

    for target in target_tags_list:
        try:
            crisis_type = correspondance_dataset[correspondance_dataset['symptome-en-simple'] == target]['type of crisis'].iloc[0]
            if crisis_type not in crisis_type_list:
                crisis_type_list.append(crisis_type)
        except:
            pass
    target_tags_list.remove(nan)
    return crisis_type_list

In [288]:
targets_list = ['atonic','central' ]

In [285]:
crisis_type_correspondance(targets_list, data_neutral)

['Generalized', 'focal']

In [286]:
html_decorate_tag_list(crisis_type_correspondance(targets_list, data_neutral))

'<span style="background-color: #DDDDDD; font-weight: 500;">Generalized</span>, <span style="background-color: #DDDDDD; font-weight: 500;">focal</span>'